In [1]:
import re
import pandas as pd
import numpy as np

# from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report


import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
#  Sequential,
from tensorflow.keras.layers import Input, Dense, LSTM, Conv2D, AveragePooling2D, Embedding, LSTM, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
#  GRU, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

# from transformers import BertTokenizerFast, TFBertModel
# from sentence_transformers import SentenceTransformer
# from transformers import DistilBertTokenizer, TFDistilBertModel

from scripts.word_embeddings import load_embedding_weights

In [2]:
def create_vocabulary(sentences_tokens):
    voc = set()
    
    voc.update(['<PAD>'])
    
    for tokens in sentences_tokens:
        voc.update(tokens)
        
    voc = list(voc)
        
    word_to_id = {word: index for word, index in zip(voc, range(len(voc)))}
    id_to_word = {index: word for word, index in zip(voc, range(len(voc)))}
    
#     print(word_to_id['<PAD>'])
    
    return voc, word_to_id, id_to_word

In [3]:
def clean(word):
    return re.sub(r'[\-\(\)\?\*\\\ \'\"\.%\^,:<>|;]', '', word.lower())

In [4]:
path = '/home/petar/Fakultet/Semester 7/NLP/Datasets/fake_news/'

In [24]:
train = pd.read_csv(path + 'full_train_df.csv', index_col=0)
test = pd.read_csv(path + 'full_test_df.csv', index_col=0)

# Preprocessing

In [25]:
train['tweet_tokens'] = train['tweet'].apply(lambda sentence: sentence.split(' '))
test['tweet_tokens'] = test['tweet'].apply(lambda sentence: sentence.split(' '))

In [26]:
train['tweet_tokens'] = train['tweet_tokens'].apply(lambda sentence: [clean(word) for word in sentence])
test['tweet_tokens'] = test['tweet_tokens'].apply(lambda sentence: [clean(word) for word in sentence])

In [27]:
tmp = train['tweet_tokens'].append(test['tweet_tokens'])

/tmp/ipykernel_18501/1723939186.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = train['tweet_tokens'].append(test['tweet_tokens'])


In [28]:
tmp.index = range(len(tmp))

In [29]:
voc, word_to_id, id_to_word = create_vocabulary(tmp)

In [30]:
embeddings = load_embedding_weights(voc, 50, 'glove')

In [31]:
train['tweet_indices'] = train['tweet_tokens'].apply(lambda sentence_tokens: [word_to_id[token] for token in sentence_tokens])
test['tweet_indices'] = test['tweet_tokens'].apply(lambda sentence_tokens: [word_to_id[token] for token in sentence_tokens])

In [32]:
train = train.groupby(['user', 'label'])['tweet_indices'].agg(list).reset_index()
test = test.groupby(['user', 'label'])['tweet_indices'].agg(list).reset_index()

In [18]:
# train['tweet_indices'] = [pad_sequences(x, value=word_to_id['<PAD>'], maxlen=100) for x in train['tweet_indices']]
# test['tweet_indices'] = [pad_sequences(x, value=word_to_id['<PAD>'], maxlen=100) for x in test['tweet_indices']]

In [33]:
tweet_indices = train['tweet_indices'].copy()
y = train['label'].copy()
# y = y.apply(lambda x: [x])

In [34]:
padded_x = [pad_sequences(x, value=word_to_id['<PAD>'], maxlen=100) for x in tweet_indices]

In [35]:
padded_x = pd.Series(padded_x)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(padded_x, y, test_size=.1, random_state=0, shuffle=True, stratify=y)

In [ ]:
# x, y = shuffle(padded_x, y, random_state=0)

In [37]:
x_train = x_train.append(x_test)
y_train = y_train.append(y_test)

/tmp/ipykernel_18501/4134551260.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_train = x_train.append(x_test)
/tmp/ipykernel_18501/4134551260.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_test)


In [38]:
tweet_indices = test['tweet_indices'].copy()
y = test['label'].copy()
# y = y.apply(lambda x: [x])

In [39]:
padded_x = [pad_sequences(x, value=word_to_id['<PAD>'], maxlen=100) for x in tweet_indices]

In [40]:
padded_x = pd.Series(padded_x)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(padded_x, y, test_size=.1, random_state=0, shuffle=True, stratify=y)

In [42]:
x_test = x_train.append(x_test)
y_test = y_train.append(y_test)

/tmp/ipykernel_18501/1631974232.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_test = x_train.append(x_test)
/tmp/ipykernel_18501/1631974232.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_test = y_train.append(y_test)


In [43]:
embeddings = embeddings.astype(np.float32)

In [44]:
x_train_tmp = x_train.to_list()
y_train_tmp = y_train.to_list()

x_test_tmp = x_test.to_list()
y_test_tmp = y_test.tolist()

# Models

In [45]:
model1 = Sequential()

model1.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], weights=[embeddings], trainable=True))

model1.add(Conv2D(128, 32, padding='valid', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model1.add(GlobalAveragePooling2D())



model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))


model1.compile(optimizer=Adam(learning_rate=.00001), loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()

2022-06-24 15:35:05.283902: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-24 15:35:05.284015: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (petar-X580VD): /proc/driver/nvidia/version does not exist
2022-06-24 15:35:05.286182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, None, 50)

In [190]:
model1.fit(x_test_tmp, y_test_tmp[0], epochs=20, batch_size=32, verbose=2)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), <class 'int'>